In [1]:
%%capture
# We need to alter the working directory since the scripts are written expecting that ./time_to_failure will be the working directory
%cd ..

In [2]:
import pandas as pd
import os
import json
from typing import List
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from data_processing import DataProcessor

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
def get_cols_with_missings(df: pd.DataFrame) -> List:
    """Returns a list of columns in a Pandas DataFrame with missing values

    Args:
        df (pd.DataFrame): input DataFrame

    Returns:
        List: list of columns with >=1 missing value
    """
    missing_share =  df.isnull().mean().reset_index()
    return missing_share[missing_share[0] > 0]['index'].tolist()

# Data Download

Data as uncompressed CSVs can be downloaded [here](https://stockholmuniversity.app.box.com/s/anmg5k93pux5p6decqzzwokp9vuzmdkh). As explained in the corresponding [white paper](https://arxiv.org/pdf/2401.15199) the data are subset into train, validation, and test splits, where only the training data contain complete information on a vehicle's entire opertional history. We therefore restrict to just the training data.

We downloaded the data locally and performed some simple merges and analyses locally.

# Read Data

In [3]:
parent_dir = '/Users/josephking/Downloads/scania_data'

train_operational = pd.read_csv(os.path.join(parent_dir, 'train_operational_readouts.csv'))
train_specifications = pd.read_csv(os.path.join(parent_dir, 'train_specifications.csv'))
train_tte = pd.read_csv(os.path.join(parent_dir, 'train_tte.csv'))

# Exploratory Data Analysis

### Operational Readouts

In [4]:
print(train_operational.shape)
train_operational.head()

(1122452, 107)


,vehicle_id,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,167_4,167_5,167_6,167_7,167_8,167_9,309_0,272_0,272_1,272_2,272_3,272_4,272_5,272_6,272_7,272_8,272_9,835_0,370_0,291_0,291_1,291_2,291_3,291_4,291_5,291_6,291_7,291_8,291_9,291_10,158_0,158_1,158_2,158_3,158_4,158_5,158_6,158_7,158_8,158_9,100_0,459_0,459_1,459_2,459_3,459_4,459_5,459_6,459_7,459_8,459_9,459_10,459_11,459_12,459_13,459_14,459_15,459_16,459_17,459_18,459_19,397_0,397_1,397_2,397_3,397_4,397_5,397_6,397_7,397_8,397_9,397_10,397_11,397_12,397_13,397_14,397_15,397_16,397_17,397_18,397_19,397_20,397_21,397_22,397_23,397_24,397_25,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
0,0,11.2,167985.0,10787.0,7413813.0,2296.0,4110.0,1296420.0,1628265.0,630345.0,1269525.0,4772940.0,2706706.0,222225.0,6240.0,0.0,70.0,1435083.0,857662.0,384579.0,668642.0,7239843.0,398490.0,3887.0,0.0,0.0,0.0,8036751.0,0.0,1227.0,555.0,463.0,925.0,468.0,225.0,535.0,516.0,492.0,729.0,66.0,97056.0,2690052.0,2945268.0,788437.0,687480.0,595164.0,491232.0,532932.0,809628.0,505693.0,858410.0,203.676778,111.911500,147.265389,200.479944,230.306278,277.722417,315.748806,372.164528,864.246250,920.881111,637.901639,744.618944,880.866889,1272.323972,1847.623667,940.785694,2.900083,0.208444,0.056417,0.058444,446956.0,411420.0,203024.0,26636.0,29156.0,7616.0,449537.0,233352.0,139920.0,12648.0,2813.0,224.0,53161.0,178881.0,138250.0,13328.0,3581.0,88.0,16361.0,131601.0,116541.0,13506.0,2856.0,48.0,6337.0,105412.0,95728.0,15609.0,1984.0,8.0,784.0,150228.0,261904.0,93172.0,17874.0,452.0
1,0,11.4,167985.0,10787.0,7413813.0,2296.0,4111.0,1302855.0,1628265.0,630345.0,1269526.0,4772940.0,2706706.0,222225.0,6240.0,0.0,70.0,1440661.0,857662.0,384579.0,668642.0,7239843.0,398490.0,3887.0,0.0,0.0,0.0,8040811.0,0.0,1230.0,558.0,463.0,925.0,469.0,226.0,535.0,516.0,493.0,729.0,66.0,97056.0,2693100.0,2947368.0,788437.0,687480.0,595164.0,491232.0,532932.0,809628.0,505693.0,860571.0,204.256750,112.924250,147.265389,201.479944,230.306278,277.722417,315.748806,372.164528,864.246250,920.881111,637.901639,745.618944,880.866889,1272.323972,1847.623667,940.785694,2.900083,0.208444,0.056417,0.058444,446964.0,411420.0,203027.0,26638.0,29157.0,7616.0,451193.0,233354.0,139920.0,12649.0,2813.0,224.0,53210.0,178883.0,138252.0,13328.0,3582.0,88.0,16368.0,131601.0,116542.0,13507.0,2856.0,48.0,6339.0,105413.0,95729.0,15610.0,1984.0,8.0,784.0,150228.0,261905.0,93172.0,17874.0,452.0
2,0,19.6,331635.0,14525.0,13683604.0,2600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,1787736.0,1133132.0,598351.0,1167062.0,12314224.0,460240.0,3887.0,0.0,0.0,0.0,12777022.0,0.0,2136.0,954.0,850.0,1420.0,722.0,412.0,880.0,666.0,586.0,1143.0,162.0,181632.0,4249020.0,4630440.0,1539133.0,1421172.0,1039764.0,749472.0,740724.0,995796.0,574045.0,1379191.0,321.671972,157.312500,193.792833,263.577611,310.711861,366.149250,415.642472,484.391167,1146.111611,1286.536333,900.062917,1123.232556,1449.545611,2140.037472,5046.748278,1151.010139,3.320194,0.218806,0.056778,0.058444,756665.0,647348.0,286811.0,30967.0,31213.0,7745.0,633790.0,423395.0,271940.0,16190.0,3573.0,232.0,75038.0,352791.0,327992.0,17325.0,4451.0,92.0,24028.0,234737.0,216619.0,17000.0,3476.0,48.0,12055.0,167693.0,142900.0,19263.0,2441.0,12.0,1420.0,204832.0,313485.0,106464.0,19306.0,452.0
3,0,20.2,354975.0,15015.0,14540449.0,2616.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,1824409.0,1166074.0,634595.0,1233908.0,13275730.0,466753.0,3887.0,0.0,0.0,0.0,13612083.0,0.0,2218.0,1014.0,892.0,1471.0,749.0,425.0,901.0,702.0,589.0,1197.0,174.0,193728.0,4462548.0,4988028.0,1696022.0,1565484.0,1112544.0,789228.0,774588.0,1015104.0,576901.0,1428606.0,331.479028,162.731639,198.104472,269.712889,320.087333,377.478667,425.901361,495.749583,1173.882583,1323.460972,923.099361,1161.893139,1501.973944,2208.782833,5587.856667,1160.593833,3.336417,0.218806,0.056778,0.058444,812577.0,686860.0,302955.0,31927.0,31488.0,7749.0,651902.0,478279.0,292109.0,16755.0,3753.0,232.0,77118.0,39408

In [5]:
train_operational.describe()

,vehicle_id,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,167_4,167_5,167_6,167_7,167_8,167_9,309_0,272_0,272_1,272_2,272_3,272_4,272_5,272_6,272_7,272_8,272_9,835_0,370_0,291_0,291_1,291_2,291_3,291_4,291_5,291_6,291_7,291_8,291_9,291_10,158_0,158_1,158_2,158_3,158_4,158_5,158_6,158_7,158_8,158_9,100_0,459_0,459_1,459_2,459_3,459_4,459_5,459_6,459_7,459_8,459_9,459_10,459_11,459_12,459_13,459_14,459_15,459_16,459_17,459_18,459_19,397_0,397_1,397_2,397_3,397_4,397_5,397_6,397_7,397_8,397_9,397_10,397_11,397_12,397_13,397_14,397_15,397_16,397_17,397_18,397_19,397_20,397_21,397_22,397_23,397_24,397_25,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
count,1.122452e+06,1.122452e+06,1.122452e+06,1.122412e+06,1.116047e+06,1.122413e+06,1.119590e+06,1.119590e+06,1.119590e+06,1.119590e+06,1.119590e+06,1.119590e+06,1.119590e+06,1.119590e+06,1.119590e+06,1.119590e+06,1.122418e+06,1.121927e+06,1.121927e+06,1.121927e+06,1.121927e+06,1.121927e+06,1.121927e+06,1.121927e+06,1.121927e+06,1.121927e+06,1.121927e+06,1.122413e+06,1.116054e+06,1.112824e+06,1.112824e+06,1.112824e+06,1.112824e+06,1.112824e+06,1.112824e+06,1.112824e+06,1.112824e+06,1.112824e+06,1.112824e+06,1.112824e+06,1.121804e+06,1.121804e+06,1.121804e+06,1.121804e+06,1.121804e+06,1.121804e+06,1.121804e+06,1.121804e+06,1.121804e+06,1.121804e+06,1.116051e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.114099e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06,1.121842e+06
mean,1.563501e+04,1.426336e+02,3.432285e+06,1.220519e+05,1.297269e+08,4.432735e+04,1.231484e+04,6.578556e+06,1.647673e+07,1.348033e+07,3.948936e+07,1.291043e+08,1.616505e+07,1.237457e+06,1.130708e+05,2.513515e+04,1.160704e+04,9.095183e+06,8.871166e+06,6.888060e+06,3.545662e+07,1.335260e+08,5.968650e+06,5.303856e+04,2.408601e+03,4.547189e+02,2.698576e+02,1.461318e+08,6.571623e+06,2.458236e+04,6.254047e+04,1.036058e+06,1.963723e+04,6.725472e+03,4.608888e+03,8.919470e+03,6.189571e+03,5.234454e+03,1.506462e+04,1.904895e+04,3.089379e+06,4.828619e+07,6.459712e+07,2.484140e+07,1.706263e+07,9.259272e+06,5.636848e+06,4.753604e+06,4.354528e+06,2.592706e+06,1.425511e+07,4.258535e+03,1.169722e+03,1.173479e+03,1.380093e+03,1.625984e+03,1.849545e+03,2.204264e+03,2.558352e+03,6.355295e+03,8.025213e+03,5.032235e+03,6.009701e+03,7.153963e+03,1.093345e+04,2.816874e+04,4.207059e+04,3.864298e+03,1.220838e+01,8.265065e-01,8.791815e-01,1.144664e+07,6.736448e+06,6.528357e+06,6.912555e+05,6.375934e+05,6.056488e+04,7.364856e+06,4.162901e+06,7.306592e+06,9.836996e+05,3.378445e+05,1.300131e+06,1.406516e+06,2.456044e+06,7.839299e+06,4.779283e+05,1.294769e+05,2.820697e+03,2.218021e+05,1.542952e+06,2.905751e+06,2.792170e+05,7.196996e+05,1.341793e+06,1.322352e+05,8.998455e+05,1.521964e+06,1.536442e+05,1.367280e+06,2.570485e+03,1.823138e+04,6.542113e+05,1.526108e+06,4.824174e+05,1.583489e+05,2.484948e+03
std,9.802149e+03,9.529169e+01,2.766660e+06,1.441444e+05,1.005308e+08,7.982317e+04,4.523900e+04,5.408065e+06,1.481707e+07,1.463525e+07,4.479542e+07,1.038244e+08,2.184020e+07,2.920862e+06,5.244031e+05,2.074423e+05,6.885284e+04,8.015971e+06,8.921667e+06,1.069562e+07,6.965757e+07,1.192697e+08,1.142653e+07,3.013526e+05,3.511383e+04,7.235416e+03,4.293170e+03,1.113199e+08,1.903266e+07,4.724309e+05,5.170522e+06,1.094849e+08,2.431065e+04,1.163433e+04,1.645840e+04,7.486330e+04,1.478193e+05,2.450421e+05,7.350800e+05,1.83640

In [6]:
pd.DataFrame(train_operational.nunique()).T

,vehicle_id,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,167_4,167_5,167_6,167_7,167_8,167_9,309_0,272_0,272_1,272_2,272_3,272_4,272_5,272_6,272_7,272_8,272_9,835_0,370_0,291_0,291_1,291_2,291_3,291_4,291_5,291_6,291_7,291_8,291_9,291_10,158_0,158_1,158_2,158_3,158_4,158_5,158_6,158_7,158_8,158_9,100_0,459_0,459_1,459_2,459_3,459_4,459_5,459_6,459_7,459_8,459_9,459_10,459_11,459_12,459_13,459_14,459_15,459_16,459_17,459_18,459_19,397_0,397_1,397_2,397_3,397_4,397_5,397_6,397_7,397_8,397_9,397_10,397_11,397_12,397_13,397_14,397_15,397_16,397_17,397_18,397_19,397_20,397_21,397_22,397_23,397_24,397_25,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
0,23550,2499,489765,334748,1102424,195376,60125,1012699,1053628,1029759,1061733,1069558,949921,558864,195480,78095,88522,1048989,1028813,995930,1054438,1069637,860299,145336,11989,6931,5580,1113460,322120,102100,77893,70518,88979,41597,31952,50480,34366,24848,41445,12252,891010,1104381,1105852,1078935,1063281,1033672,998015,980513,956297,699768,1088038,1096660,1091997,1089175,1086439,1083619,1080564,1078242,1076985,1076475,1075358,1073397,1072428,1071436,1070200,1069234,1062603,967585,540134,176709,87479,1089190,1073256,1060520,712166,547194,223257,1067078,1030577,1072353,450185,202569,39618,785524,982662,1065138,429656,201559,26643,445901,917353,1007391,391415,172182,22199,332871,840061,930447,368960,163267,15502,82755,731125,902735,657916,353455,36607


In [7]:
pd.DataFrame(train_operational.isnull().mean()).T

,vehicle_id,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,167_4,167_5,167_6,167_7,167_8,167_9,309_0,272_0,272_1,272_2,272_3,272_4,272_5,272_6,272_7,272_8,272_9,835_0,370_0,291_0,291_1,291_2,291_3,291_4,291_5,291_6,291_7,291_8,291_9,291_10,158_0,158_1,158_2,158_3,158_4,158_5,158_6,158_7,158_8,158_9,100_0,459_0,459_1,459_2,459_3,459_4,459_5,459_6,459_7,459_8,459_9,459_10,459_11,459_12,459_13,459_14,459_15,459_16,459_17,459_18,459_19,397_0,397_1,397_2,397_3,397_4,397_5,397_6,397_7,397_8,397_9,397_10,397_11,397_12,397_13,397_14,397_15,397_16,397_17,397_18,397_19,397_20,397_21,397_22,397_23,397_24,397_25,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
0,0.0,0.0,0.0,0.000036,0.005706,0.000035,0.00255,0.00255,0.00255,0.00255,0.00255,0.00255,0.00255,0.00255,0.00255,0.00255,0.00003,0.000468,0.000468,0.000468,0.000468,0.000468,0.000468,0.000468,0.000468,0.000468,0.000468,0.000035,0.0057,0.008578,0.008578,0.008578,0.008578,0.008578,0.008578,0.008578,0.008578,0.008578,0.008578,0.008578,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.005703,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.007442,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543,0.000543


### Specifications

In [8]:
print(train_specifications.shape)
train_specifications.head()

(23550, 9)


,vehicle_id,Spec_0,Spec_1,Spec_2,Spec_3,Spec_4,Spec_5,Spec_6,Spec_7
0,0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0
1,2,Cat0,Cat1,Cat1,Cat0,Cat0,Cat0,Cat0,Cat1
2,3,Cat0,Cat1,Cat1,Cat1,Cat0,Cat0,Cat0,Cat1
3,4,Cat0,Cat0,Cat2,Cat1,Cat0,Cat0,Cat0,Cat1
4,5,Cat0,Cat2,Cat2,Cat0,Cat0,Cat0,Cat0,Cat1


In [9]:
train_specifications.nunique()

vehicle_id    23550
Spec_0            3
Spec_1           29
Spec_2           21
Spec_3            4
Spec_4            2
Spec_5            5
Spec_6           17
Spec_7            9
dtype: int64

In [10]:
train_specifications.isnull().mean()

vehicle_id    0.0
Spec_0        0.0
Spec_1        0.0
Spec_2        0.0
Spec_3        0.0
Spec_4        0.0
Spec_5        0.0
Spec_6        0.0
Spec_7        0.0
dtype: float64

### TTE

In [11]:
print(train_tte.shape)
train_tte.head()

(23550, 3)


,vehicle_id,length_of_study_time_step,in_study_repair
0,0,510.0,0
1,2,281.8,0
2,3,293.4,0
3,4,210.0,0
4,5,360.4,0


In [12]:
train_tte.describe()

,vehicle_id,length_of_study_time_step,in_study_repair
count,23550.000000,23550.000000,23550.000000
mean,16851.663185,240.349019,0.096476
std,9714.309420,88.782437,0.295249
min,0.000000,73.400000,0.000000
25%,8468.250000,163.800000,0.000000
50%,16882.500000,218.200000,0.000000
75%,25247.750000,312.000000,0.000000
max,33643.000000,510.000000,1.000000


In [13]:
train_tte.isnull().mean()

vehicle_id                   0.0
length_of_study_time_step    0.0
in_study_repair              0.0
dtype: float64

### TTE lifespan versus max observed lifespan in `train_operational`

As we see below, the last actual observation is typically about 5.6 timestamps later than the last observation in the `train_operational` dataset. There are no feature values at the time of actual last observation.

In [14]:
max_time_step = train_operational.groupby('vehicle_id')['time_step'].transform('max')
last_obs = train_operational[train_operational['time_step'] == max_time_step][['vehicle_id', 'time_step']]
last_obs = pd.merge(last_obs, train_tte, on='vehicle_id')

failed = last_obs[last_obs['in_study_repair']==1]
no_fail = failed = last_obs[last_obs['in_study_repair']==0]


In [15]:
(last_obs['length_of_study_time_step'] - last_obs['time_step']).describe()

count    23550.000000
mean         5.588365
std         11.877423
min          0.200000
25%          0.800000
50%          2.000000
75%          5.000000
max        324.600000
dtype: float64

In [16]:
(failed['length_of_study_time_step'] - failed['time_step']).describe()

count    21278.000000
mean         5.771510
std         11.875166
min          0.200000
25%          0.800000
50%          2.200000
75%          5.200000
max        324.600000
dtype: float64

In [17]:
(no_fail['length_of_study_time_step'] - no_fail['time_step']).describe()

count    21278.000000
mean         5.771510
std         11.875166
min          0.200000
25%          0.800000
50%          2.200000
75%          5.200000
max        324.600000
dtype: float64

# Merge

In [18]:
df = pd.merge(train_operational, train_specifications, on='vehicle_id')

### Merge using the last timestamp contained in `train_operational`, treat this as failure/censored timepoint

In [19]:
df = pd.merge(df, last_obs[['vehicle_id', 'time_step', 'in_study_repair']], on=['vehicle_id', 'time_step'], how='left')

### Fill previous panel observations with 0 for `in_study_repair`

In [20]:
df['in_study_repair'] = df['in_study_repair'].fillna(0).astype(int)

### Rearrange columns

In [21]:
key_cols = ['vehicle_id', 'time_step', 'in_study_repair']
rest_cols = sorted([i for i in df.columns if i not in key_cols])
new_col_ordeer = key_cols + rest_cols

In [22]:
df = df[new_col_ordeer]

In [23]:
df = df.sort_values(by=['vehicle_id', 'time_step']).reset_index(drop=True)

In [24]:
df

,vehicle_id,time_step,in_study_repair,100_0,158_0,158_1,158_2,158_3,158_4,158_5,158_6,158_7,158_8,158_9,167_0,167_1,167_2,167_3,167_4,167_5,167_6,167_7,167_8,167_9,171_0,272_0,272_1,272_2,272_3,272_4,272_5,272_6,272_7,272_8,272_9,291_0,291_1,291_10,291_2,291_3,291_4,291_5,291_6,291_7,291_8,291_9,309_0,370_0,397_0,397_1,397_10,397_11,397_12,397_13,397_14,397_15,397_16,397_17,397_18,397_19,397_2,397_20,397_21,397_22,397_23,397_24,397_25,397_26,397_27,397_28,397_29,397_3,397_30,397_31,397_32,397_33,397_34,397_35,397_4,397_5,397_6,397_7,397_8,397_9,427_0,459_0,459_1,459_10,459_11,459_12,459_13,459_14,459_15,459_16,459_17,459_18,459_19,459_2,459_3,459_4,459_5,459_6,459_7,459_8,459_9,666_0,835_0,837_0,Spec_0,Spec_1,Spec_2,Spec_3,Spec_4,Spec_5,Spec_6,Spec_7
0,0,11.2,0,858410.0,97056.0,2690052.0,2945268.0,788437.0,687480.0,595164.0,491232.0,532932.0,809628.0,505693.0,4110.0,1296420.0,1628265.0,630345.0,1269525.0,4772940.0,2706706.0,222225.0,6240.0,0.0,167985.0,1435083.0,857662.0,384579.0,668642.0,7239843.0,398490.0,3887.0,0.0,0.0,0.0,1227.0,555.0,66.0,463.0,925.0,468.0,225.0,535.0,516.0,492.0,729.0,70.0,0.0,446956.0,411420.0,2813.0,224.0,53161.0,178881.0,138250.0,13328.0,3581.0,88.0,16361.0,131601.0,203024.0,116541.0,13506.0,2856.0,48.0,6337.0,105412.0,95728.0,15609.0,1984.0,8.0,26636.0,784.0,150228.0,261904.0,93172.0,17874.0,452.0,29156.0,7616.0,449537.0,233352.0,139920.0,12648.0,7413813.0,203.676778,111.911500,637.901639,744.618944,880.866889,1272.323972,1847.623667,940.785694,2.900083,0.208444,0.056417,0.058444,147.265389,200.479944,230.306278,277.722417,315.748806,372.164528,864.246250,920.881111,10787.0,8036751.0,2296.0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0
1,0,11.4,0,860571.0,97056.0,2693100.0,2947368.0,788437.0,687480.0,595164.0,491232.0,532932.0,809628.0,505693.0,4111.0,1302855.0,1628265.0,630345.0,1269526.0,4772940.0,2706706.0,222225.0,6240.0,0.0,167985.0,1440661.0,857662.0,384579.0,668642.0,7239843.0,398490.0,3887.0,0.0,0.0,0.0,1230.0,558.0,66.0,463.0,925.0,469.0,226.0,535.0,516.0,493.0,729.0,70.0,0.0,446964.0,411420.0,2813.0,224.0,53210.0,178883.0,138252.0,13328.0,3582.0,88.0,16368.0,131601.0,203027.0,116542.0,13507.0,2856.0,48.0,6339.0,105413.0,95729.0,15610.0,1984.0,8.0,26638.0,784.0,150228.0,261905.0,93172.0,17874.0,452.0,29157.0,7616.0,451193.0,233354.0,139920.0,12649.0,7413813.0,204.256750,112.924250,637.901639,745.618944,880.866889,1272.323972,1847.623667,940.785694,2.900083,0.208444,0.056417,0.058444,147.265389,201.479944,230.306278,277.722417,315.748806,372.164528,864.246250,920.881111,10787.0,8040811.0,2296.0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0
2,0,19.6,0,1379191.0,181632.0,4249020.0,4630440.0,1539133.0,1421172.0,1039764.0,749472.0,740724.0,995796.0,574045.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,331635.0,1787736.0,1133132.0,598351.0,1167062.0,12314224.0,460240.0,3887.0,0.0,0.0,0.0,2136.0,954.0,162.0,850.0,1420.0,722.0,412.0,880.0,666.0,586.0,1143.0,70.0,0.0,756665.0,647348.0,3573.0,232.0,75038.0,352791.0,327992.0,17325.0,4451.0,92.0,24028.0,234737.0,286811.0,216619.0,17000.0,3476.0,48.0,12055.0,167693.0,142900.0,19263.0,2441.0,12.0,30967.0,1420.0,204832.0,313485.0,106464.0,19306.0,452.0,31213.0,7745.0,633790.0,423395.0,271940.0,16190.0,13683604.0,321.671972,157.312500,900.062917,1123.232556,1449.545611,2140.037472,5046.748278,1151.010139,3.320194,0.218806,0.056778,0.058444,193.792833,263.577611,310.711861,366.149250,415.642472,484.391167,1146.111611,1286.536333,14525.0,12777022.0,2600.0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0
3,0,20.2,0,1428606.0,193728.0,4462548.0,4988028.0,1696022.0,1565484.0,1112544.0,789228.0,774588.0,1015104.0,576901.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,354975.0,1824409.0,1166074.0,634595.0,1233908.0,13275730.0,466753.0,3887.0,0.0,0.0,0.0,2218.0,1014.0,174.0,892.0,1471.0,749.0,425.0,901.0,702.0,589.0,1197.0,70.0,0.0,812577.0,686860.0,3753.0,232.0,77118.0,394083.0,359060.0,17941.0,4573.0,92.0,25164.0,253706.0,302955.0,232912.0,17583.0,3573.0,48.0,13199.0,176596.0,150565.0,19832.0,2

# Output Combined (Raw) Dataset

This dataset is a combination of: 1) operational; 2) specifications; 3) Tte. 

In [25]:
df.to_csv('./data/raw_scania.csv.gz', compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1}, index=False)

# Impute Missings

### One-hot encode categorical features

In [26]:
# One-hot encoding of categorical features
df_ = pd.get_dummies(df, drop_first=True, dtype=int)

### Impute missings using groupby forward fill

In [27]:
# Fill missings
cols_missing_data = get_cols_with_missings(df_)
for col in cols_missing_data:  # forward fill, grouped by `vehicle_id`
    df_[col] = df_.groupby('vehicle_id')[col].transform('ffill')

### [MICE Imputation](https://medium.com/@kunalshrm175/multivariate-imputation-by-chained-equations-mice-2d3efb063434) using sklearn's [IterativeImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html), with [Bayesian ridige regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html) under-the-hood

In [28]:
imp = IterativeImputer(
    max_iter = 5,
    tol = 1e-10,
    imputation_order = 'roman',
    random_state = 42,
    initial_strategy = 'mean',
    verbose = 2,
    skip_complete=True
)

df_ = pd.DataFrame(imp.fit_transform(df_), index=df_.index, columns=df_.columns)

[IterativeImputer] Completing matrix with shape (1122452, 190)
[IterativeImputer] Ending imputation round 1/5, elapsed time 1147.64
[IterativeImputer] Change: 331846929.4486323, scaled tolerance: 3.4225874055000003 
[IterativeImputer] Ending imputation round 2/5, elapsed time 2262.16
[IterativeImputer] Change: 452899035.4151731, scaled tolerance: 3.4225874055000003 
[IterativeImputer] Ending imputation round 3/5, elapsed time 3365.85
[IterativeImputer] Change: 68021980.05074555, scaled tolerance: 3.4225874055000003 
[IterativeImputer] Ending imputation round 4/5, elapsed time 4446.31
[IterativeImputer] Change: 33423163.95587253, scaled tolerance: 3.4225874055000003 
[IterativeImputer] Ending imputation round 5/5, elapsed time 5548.94
[IterativeImputer] Change: 26982790.090393487, scaled tolerance: 3.4225874055000003 


/Users/josephking/miniconda3/envs/time_to_failure/lib/python3.10/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


### Remove on-hot encoded cols and re-attach original columns

In [11]:
non_cat_cols = [i for i in df_.columns if "Spec" not in i]
cat_cols = [i for i in df.columns if "Spec" in i]

In [12]:
imputed = pd.concat([df_[non_cat_cols], df[cat_cols]], axis=1)

In [14]:
imputed.isnull().mean().mean()

np.float64(0.0)

In [15]:
imputed.shape

(1122452, 116)

# Output Imputed (Raw) Dataset

These data are still raw data, no data transformations have been applied

In [16]:
imputed.to_csv('./data/imputed_scania.csv.gz', compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1}, index=False)